In [1]:
#Importamos los paquetes necesarios para el funcionamiento.
import csv
import tensorflow
tensorflow.config.run_functions_eagerly(False)

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.layers import Conv1D, GlobalMaxPooling1D
from tensorflow.keras.datasets import imdb
from tqdm import tqdm
import requests

In [ ]:
#Usaremos colab para cargar los documentos de la tarea 1.
from google.colab import drive

drive.mount('/content/gdrive')

In [ ]:
"""
Método add_feature1
Toma una matriz dispera x y agrega una o más características adicionales.
@param: dos matrices en formato csv.
@return: una matriz en formato csr.
"""
def add_feature1(X, feature_to_add):
    from scipy.sparse import csr_matrix, hstack
    # Se concatena la secuncia de caracteres del conjunto de train con un matriz dispersa
    return hstack([X, csr_matrix(feature_to_add)], 'csr')

"""
Método plot_graphs
Genera gráficos qeu muestra la evolución de métricas como pérdida o precisió, tanto en los datos de entrenamiento como de validación.
@param: history -> la historia del modelo, string -> el nombre de la métrica a graficar.
@return: Gráfico
"""
def plot_graphs(history, string):
    plt.plot(history.history[string])
    plt.plot(history.history['val_'+string])
    plt.xlabel("Epochs")
    plt.ylabel(string)
    plt.legend([string, 'val_'+string])
    plt.show()

In [ ]:
"""
Obetenemos los datos de entrenamiento y validación ya refinados.
"""
#Dataset roberta finetuned
train_roberta = pd.read_csv('/content/gdrive/MyDrive/subtask_1/train_subtask1xlm-roberta-base-finetuned-IberAuTexTification2024-7030-4epo-task1-v2.csv', header=None)
test_roberta = pd.read_csv('/content/gdrive/MyDrive/subtask_1/test_subtask1xlm-roberta-base-finetuned-IberAuTexTification2024-7030-4epo-task1-v2.csv', header=None)

#Dataset bert
train_bert = pd.read_csv('/content/gdrive/MyDrive/subtask_1/train_subtask1bert-base-multilingual-cased-finetuned-autext24.csv', header=None)
test_bert = pd.read_csv('/content/gdrive/MyDrive/subtask_1/test_subtask1bert-base-multilingual-cased-finetuned-autext24.csv', header=None)

#Dataset e5
train_e5 = pd.read_csv('/content/gdrive/MyDrive/subtask_1/train_subtask1multilingual-e5-large-finetuned-autext24.csv', header=None)
test_e5 = pd.read_csv('/content/gdrive/MyDrive/subtask_1/test_subtask1multilingual-e5-large-finetuned-autext24.csv', header=None)

#Dataset stylometry
train_stylometry = pd.read_csv('/content/gdrive/MyDrive/subtask_1/stylometry_train_S1.csv')
test_stylometry = pd.read_csv('/content/gdrive/MyDrive/subtask_1/stylometry_test_S1.csv')


In [ ]:
#Dataset original
train_data = pd.read_csv('/content/gdrive/MyDrive/subtask_1/train_S1.csv')
test_data = pd.read_csv('/content/gdrive/MyDrive/subtask_1/test_S1.csv')

#Convertimos la columna "label" en etiquetas binarias, si el valor es "generated", le dará el valor de 1 o de lo contrario será 0.
train_data['label'] = np.where(train_data['label']=='generated',1,0)
test_data['label'] = np.where(test_data['label']=='generated',1,0)

#A la columna text del conjunto se le asignan las características independientes, mientras que a la columna "label" la etiqueta.
X_train_data = train_data['text']
y_train_data = train_data['label']

#A la columna text del conjunto se le asignan las características independientes, mientras que a la columna "label" la etiqueta.
X_test_data = test_data['text']
y_test_data = test_data['label']

#Mostramos las dimensiones (número de filas y columnas) de los datos de entrenamiento y de validación.
print(train_data.shape)
print(test_data.shape)

In [ ]:
#Datos de entrenamiento
#Unimos todas las características para el entrenamiento en un solo objeto.
X_train_data = add_feature1(train_stylometry, train_bert)
X_train_data = add_feature1(X_train, train_e5)
X_train_data = add_feature1(X_train, train_roberta)

#Datos de prueba
#Unimos todas las características oara la validación en un solo objeto.
X_test_data = add_feature1(test_roberta, test_bert)
X_test_data = add_feature1(X_test, test_e5)
X_test_data = add_feature1(X_test, test_stylometry)

In [ ]:
#Volvemos a mostrar las dimensiones de las características viendo que coincidan en el número de filas
print(X_train_data.shape)
print(y_train_data.shape)

In [ ]:
"""
Comenzamos el procseo de la Red Neuronal simple.
Con la primera capa densa conectada con 128 neuronas y la segundo con 64 con la función ReLu que introduce la no linealiedad, además que X_train_data.shape[1] defina las entradas en la red.
El primer y segundo dropout que desactiva el 20% de las neuronas para prevenir el sobreajuste.
Finalmente con una capa de salida con la función sigmoid debido a que estamos haciendo una clasificación binaria.
Si fuera el task 2, se usariía softmax para esta última capa ya que son más de dos categorías de salida.
"""
model = tensorflow.keras.Sequential([
    tensorflow.keras.layers.Dense(128, activation='relu', input_shape = (X_train_data.shape[1],)),
    tensorflow.keras.layers.Dropout(0.2),
    tensorflow.keras.layers.Dense(64, activation='relu'),
    tensorflow.keras.layers.Dropout(0.2),
    tensorflow.keras.layers.Dense(1, activation='sigmoid')
])

"""
Creamos el optimizador Adam con un learning rate de 0.001 que es lo que controla el tamali de los pasos que toma durante la actualización.
"""
learning_rate = 0.001
optimizer = tensorflow.keras.optimizers.Adam(learning_rate=learning_rate)

"""
Compilamos el modelo con la función de pérdida binary_crossentropy, el optimizador Adam y la métrica de precisión, que mide el rendimiento del modelo.
"""
model.compile(loss='binary_crossentropy', optimizer= optimizer, metrics=['accuracy'])
model.summary() #Mostramos el modelo

In [ ]:
"""
Se entrena el modelo con los datos proporcionados, donde history almacena el historial del entrenamiento.
@X_train_data: Datos de entrada.
@y_train_data: Datos de salida correpondientes a X_train_data.
@epochs: El número de veces que el modelo pasrá por el conjunto, entre más mejor.
@batch_size: Define el tamaño en el que los datos se dividirán para el entrenamiento.
@validation_split: Define la fracción de los datos de entrenamiento que se utilizarán para la validación.
"""
history = model.fit(X_train_data, y_train_data, epochs=5, batch_size=128, validation_split=0.3)

In [ ]:
"""
Usamos el método anteriormente creado.
Donde accurcay represneta la precisión de los datos de entrenamiento.
Y val_accuracy representa la precisión de los datos de validación.
"""
plt.show()

In [ ]:
#Evaluación, checamos para saber la precisión del modelo que se medirá en una probabbilidad de 0 a 1.
test_loss,test_acc = model.evaluate(X_test, y_test_data)
print('Test Accuracy: ', test_acc)